In [1]:
# Import required libraries
import yfinance as yf
import pandas as pd
import sqlite3
import os
from datetime import datetime
from tqdm.notebook import tqdm

# Define the data directory
DATA_DIR = "./../data"

### Load S&P500 stock tickers from wikipedia + get historical data from Yahoo Finance

In [2]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory


def get_and_store_data(tickers):
    valid_tickers = []

    # Ensure data directory exists
    ensure_dir(DATA_DIR)

    # Create subdirectories for SQLite and Parquet files
    parquet_dir = os.path.join(DATA_DIR, "parquet")
    ensure_dir(parquet_dir)


    # Create progress bar
    pbar = tqdm(tickers, desc="Processing Stocks", unit="stock")

    for ticker in pbar:
        try:
            # if file already exists, skip
            if os.path.exists(os.path.join(parquet_dir, f"{ticker}.parquet")):
                pbar.set_description(f"Skipping {ticker} (already processed)")
                valid_tickers.append(ticker)
                continue

            stock = yf.Ticker(ticker)

            # Get all available history
            history = stock.history(period="max")

            if not history.empty:
                valid_tickers.append(ticker)

                # Store as Parquet
                parquet_path = os.path.join(parquet_dir, f"{ticker}.parquet")
                history.to_parquet(parquet_path)

                # Update progress bar description
                pbar.set_description(
                    f"Processed {ticker} ({history.index[0].date()} to {history.index[-1].date()})"
                )
            else:
                pbar.set_description(f"No data for {ticker}")

        except Exception as e:
            pbar.set_description(
                f"Error with {ticker}: {str(e)[:50]}..."
            )  # Truncate long error messages

    return valid_tickers


# Get S&P 500 components
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500_tickers = sp500["Symbol"].tolist()

print("Starting data collection and storage process...")
valid_tickers = get_and_store_data(sp500_tickers)

print(f"\nProcess completed. Total valid tickers: {len(valid_tickers)}")
print(f"First 10 valid tickers: {valid_tickers[:10]}")


Starting data collection and storage process...


Processing Stocks:   0%|          | 0/503 [00:00<?, ?stock/s]

$BRK.B: possibly delisted; no timezone found
$BF.B: possibly delisted; no price data found  (1d 1925-09-17 -> 2024-08-23)



Process completed. Total valid tickers: 501
First 10 valid tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


### (optional) convert .parquet to sqlite for easier querying

In [3]:
# construct the SQLite database from the Parquet files

def construct_sqlite_db():
    
    tickers, db_path = valid_tickers, os.path.join(DATA_DIR,"sqlite", "stock_data.sqlite")
    
    # remove file if it already exists
    if os.path.exists(db_path):
        os.remove(db_path)
    
    # Connect to SQLite database
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    # Create table
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS stock_data (
            Ticker TEXT,
            Date TEXT,
            Open REAL,
            High REAL,
            Low REAL,
            Close REAL,
            Volume INTEGER,
            Dividends REAL,
            Stock_Splits REAL
        )
        """
    )

    # Insert data
    for ticker in tqdm(tickers, desc="Processing Stocks", unit="stock"):
        parquet_path = os.path.join(DATA_DIR, "parquet", f"{ticker}.parquet")
        stock_data = pd.read_parquet(parquet_path)

        # Add ticker column
        stock_data["Ticker"] = ticker
        
        stock_data["Stock_Splits"] = stock_data["Stock Splits"]
        stock_data.drop(columns=["Stock Splits"], inplace=True)

        # Insert into SQLite
        stock_data.to_sql("stock_data", conn, if_exists="append", index=True)

    # Commit and close
    conn.commit()
    conn.close()
    
    
construct_sqlite_db()


Processing Stocks:   0%|          | 0/501 [00:00<?, ?stock/s]

### (optional) Load data from sqlite to pandas dataframe and serialize to parquet

In [1]:
output_dir = ensure_dir(os.path.join(DATA_DIR, "data_frame"))
print(f"Output directory: {output_dir}")

df = pd.read_sql_query("SELECT * FROM stock_data", sqlite3.connect(os.path.join(DATA_DIR, "sqlite", "stock_data.sqlite")))
print(f"Number of rows in SQLite database: {len(df)}")

df.to_parquet(f"{output_dir}/stock_data.parquet")

df = pd.read_parquet(f"{output_dir}/stock_data.parquet")
print(f"Number of rows in Parquet file: {len(df)}")

NameError: name 'ensure_dir' is not defined